In [1]:
import os
from dotenv import load_dotenv
import mlflow
from mlflow_utils import create_mlflow_experiment
from mlflow.models.signature import ModelSignature
from mlflow.types import ColSpec
from mlflow.types import Schema
from mlflow.types import ParamSpec
from mlflow.types import ParamSchema

import pandas as pd
import numpy as np
from typing import Tuple

load_dotenv()

True

In [2]:
# conectar con mlflow y minio (por http)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://127.0.0.1:9000"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('ACCESS_KEY')

# Multiple models

In [3]:
class CustomModel(mlflow.pyfunc.PythonModel):
    def predict_model1(self, model_input):
        # do some processing for model 1
        return 0 * model_input

    def predict_model2(self, model_input):
        # do some processing for model 2
        return model_input

    def predict_model3(self, model_input):
        # do some processing for model 3
        return 2 * model_input

    def predict(self, context, model_input, params):
        if params["model_name"] == "model_1":
            return self.predict_model1(model_input=model_input)

        elif params["model_name"] == "model_2":
            return self.predict_model2(model_input=model_input)

        elif params["model_name"] == "model_3":
            return self.predict_model3(model_input=model_input)

        else:
            raise Exception("Model Not Found!")

In [4]:
input_schema = Schema(inputs=[ColSpec(type="integer", name="input")])
output_schema = Schema(inputs=[ColSpec(type="integer", name="output")])
param_spec = ParamSpec(name="model_name", dtype="string", default=None)
param_schema = ParamSchema(params=[param_spec])
model_signature = ModelSignature(
    inputs=input_schema, outputs=output_schema, params=param_schema
)
model_signature

inputs: 
  ['input': integer (required)]
outputs: 
  ['output': integer (required)]
params: 
  ['model_name': string (default: None)]

In [5]:
# experiment_id = create_mlflow_experiment(
#     experiment_name="Serving Multiple Models",
#     artifact_location="serving_multiple_models",
#     tags={"purpose": "learning"},
# )

mlflow.set_experiment("mlflow_tracking")

# with mlflow.start_run(run_name="multiple_models", experiment_id=experiment_id) as run:
with mlflow.start_run(run_name="logging_artifacts19") as run:

    mlflow.pyfunc.log_model(artifact_path="model", python_model=CustomModel(), signature=model_signature)

    model_uri = f"runs:/{run.info.run_id}/model"
    loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)

    for n in range(3):
        print(f"PREDICTION FROM MODEL {n+1}")
        print(loaded_model.predict(data={"input":np.int32(10)}, params={"model_name":f"model_{n+1}"}))

    print(f"RUN_ID: {run.info.run_id}")

/home/oecorrechag/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

PREDICTION FROM MODEL 1
   input
0      0
PREDICTION FROM MODEL 2
   input
0     10
PREDICTION FROM MODEL 3
   input
0     20
RUN_ID: 88d353e428ee470db959f6b6fe2152d5


In [6]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://127.0.0.1:5000
artifact uri: s3://mlflow/3/42a662ee775d4b06b9879533aae5a3ab/artifacts


In [7]:
mlflow.end_run()

In [8]:
print('ok_')

ok_
